In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import gudhi as gd
from scipy.spatial.distance import hamming, pdist, squareform
import kaleido
import plotly.graph_objs as go
import networkx as nx
import plotly.graph_objects as go
import plotly.io as pio
from scipy import sparse

plt.rcParams['font.sans-serif'] = ['Liberation Sans']

In [ ]:


def visualize_simplicial_complex(simplex_tree, filtration_value, vertex_names=None, save_filename=None, plot_size=1, dpi=600):
    G = nx.Graph()
    triangles = []
    color_map = plt.get_cmap('coolwarm')

    # Create a dictionary to map simplices to colors based on their filtration values
    simplex_colors = {}
    max_filtration = max(filt for _, filt in simplex_tree.get_filtration() if filt <= filtration_value)

    for simplex, filt in simplex_tree.get_filtration():
        if filt <= filtration_value:
            # Normalize the filtration value and get the corresponding color
            normalized_value = filt / max_filtration
            color = color_map(normalized_value)
            rgba_color = 'rgba({}, {}, {}, {})'.format(int(color[0]*255), int(color[1]*255), int(color[2]*255), color[3])
            simplex_colors[tuple(simplex)] = rgba_color

            # Add nodes and edges/triangles to the graph
            if len(simplex) == 2:
                G.add_edge(simplex[0], simplex[1])
            elif len(simplex) == 1:
                G.add_node(simplex[0])
            elif len(simplex) == 3:
                triangles.append(simplex)

    # Position nodes using NetworkX's spring layout
    pos = nx.spring_layout(G)

    # Node trace
    x_values = [pos[key][0] for key in pos]
    y_values = [pos[key][1] for key in pos]
    node_labels = [vertex_names[node] if vertex_names else node for node in pos]
    #node_trace = go.Scatter(x=x_values, y=y_values, mode='markers+text', hoverinfo="text", marker=dict(size=10), text=node_labels, textposition='top center')
    node_trace = go.Scatter(x=x_values, y=y_values, mode='markers+text', hoverinfo="text",marker=dict(size=14), text=node_labels, textposition='top center', textfont=dict(size=14))

    
    # Edge traces
    edge_traces = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        color = simplex_colors.get(edge, 'rgba(0, 0, 0, 0.5)')  # Default color if edge not in simplex_colors
        edge_trace = go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines', line=dict(width=3, color=color))
        edge_traces.append(edge_trace)

        # Triangle traces
    triangle_traces = []
    for triangle in triangles:
        x0, y0 = pos[triangle[0]]
        x1, y1 = pos[triangle[1]]
        x2, y2 = pos[triangle[2]]
    # Utilizando el nuevo color RGBA para los triángulos
        color = 'rgba(244, 157, 126, 0.545)'  # Color #F49D7E en formato RGBA
        triangle_trace = go.Scatter(x=[x0, x1, x2, x0, None], y=[y0, y1, y2, y0, None], fill='toself', mode='lines', line=dict(width=3), fillcolor=color)
        triangle_traces.append(triangle_trace)

    # Combine all traces and create the figure

    layout = go.Layout(showlegend=False, hovermode='closest', xaxis=dict(showgrid=False, zeroline=False,tickfont=dict(size=16, family='Arial, sans-serif') ), yaxis=dict(showgrid=False, zeroline=False,tickfont=dict(size=16, family='Arial, sans-serif')))

    fig = go.Figure(data=edge_traces + triangle_traces + [node_trace], layout=layout)

    # Set the figure size
    fig.update_layout(width=plot_size * dpi, height=plot_size * dpi)

    # Save the figure if a filename is provided
    if save_filename:
        pio.write_image(fig, save_filename, width=plot_size * dpi, height=plot_size * dpi, scale=1)

    # Show the figure
    fig.show()

    return G

In [ ]:
def create_complex(distance_matrix):
    # Create the Rips simplicial complex from the distance matrix
    rips_complex = gd.RipsComplex(distance_matrix=distance_matrix)
    # Create the simplex tree from the Rips complex with a maximum dimension of 3
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=3)
    # Compute the persistence of the simplicial complex
    persistence = simplex_tree.persistence()
    # Return the persistence diagram or barcode
    return persistence, simplex_tree
#### Function for visualization
def plot_dendrogram(data):
    """Plot a dendrogram from the data."""
    linked = linkage(data, 'single')
    dendrogram(linked, orientation='top', distance_sort='descending')
    plt.show()

In [ ]:
# Let's assume that "population" is a numpy ndarray with your genomes as rows.
def calculate_hamming_matrix(population):
    # Number of genomes
    num_genomes = population.shape[0]
    # Create an empty matrix for Hamming distances
    hamming_matrix = np.zeros((num_genomes, num_genomes), dtype=int)
   # Calculate the Hamming distance between each pair of genomes
    for i in range(num_genomes):
        for j in range(i+1, num_genomes):  # j=i+1 to avoid calculating the same distance twice
            # The Hamming distance is multiplied by the number of genes to convert it into an absolute distance
            distance = hamming(population[i], population[j]) * len(population[i])
            hamming_matrix[i, j] = distance
            hamming_matrix[j, i] = distance  # The matrix is symmetric
    
    return hamming_matrix
# Assuming that "population" is a NumPy ndarray with your genomes as rows
def calculate_normalized_hamming_matrix(population):
    num_genomes = population.shape[0]
    num_genes = population.shape[1]
    hamming_matrix = np.zeros((num_genomes, num_genomes), dtype=float)
    
    for i in range(num_genomes):
        for j in range(i+1, num_genomes):  # It's not necessary to calculate when i == j
            # The sum of boolean comparisons will give the total number of differences
            distance = np.sum(population[i] != population[j]) / num_genes
            hamming_matrix[i, j] = distance
            hamming_matrix[j, i] = distance  # Taking advantage of symmetry
    
    return hamming_matrix


In [ ]:
def plot_barcodes(intervals, color, ax):
    for idx, (start, end) in enumerate(intervals):
        ax.plot([start, end], [idx, idx], color=color, lw=5)
def plot_intervals_esc(bacteria, save_path=None):
    fig, ax = plt.subplots(figsize=(10, 10))

    # Grafica los códigos de barras
    #plot_barcodes(bacteria, '#F34E81', ax)
    plot_barcodes(bacteria, '#274E13', ax)

    # Remueve las etiquetas del eje y
    ax.get_yaxis().set_ticks([])

    # Añade leyendas
    #bacteria_legend = mlines.Line2D([], [], color='#F34E81', label='$Klebsiella$ $P.$ ')
    bacteria_legend = mlines.Line2D([], [], color='#274E13', label='$Escherichia$ $C.$')
    ax.legend(handles=[bacteria_legend],fontsize=16)

#     ax.set_title(r'H1 Persistence Barcode', fontsize=20)
#     ax.set_xlabel('Persistence Interval', fontsize=14)
    plt.xticks(fontsize=16)  # Ajusta el tamaño según sea necesario
    plt.yticks(fontsize=16)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    else:
        plt.show()

In [ ]:
# link="https://raw.githubusercontent.com/shadayguerrero/TDA_Horizontal_Genes_Transfer/main/data/BVBRC_protein_families_all_final.csv"
import pandas as pd

df = pd.read_csv("sphdata.dat", sep='\t', index_col=0)
df


In [ ]:
matrix=df.iloc[:,1:4].values



In [ ]:
matrix_distance=pdist(df[['x[cm]','y[cm]','z[cm]']], metric='euclidean')

In [ ]:
np.shape(matrix_distance)[0]

In [ ]:
df[['x[cm]','y[cm]','z[cm]']].iloc[:3]

In [ ]:
minimo=matrix_distance.min()
minimo

In [ ]:
maximo=matrix_distance.max()
maximo

In [ ]:
promedio=matrix_distance.mean()

In [ ]:
point_cloud = df.iloc[:,1:4].values.tolist()
point_cloud

In [ ]:
alpha_complex=gd.AlphaComplex(points=point_cloud)

In [ ]:


# Crear el complejo simplicial filtrado
simplex_tree = alpha_complex.create_simplex_tree()

# Calcular la homología persistente
diag = simplex_tree.persistence()

# Mostrar los diagramas de persistencia
gd.plot_persistence_diagram(diag)

In [ ]:
persistence_dim_0=[p for p in diag if p[0]==0]
gd.plot_persistence_barcode(persistence_dim_0)

In [ ]:
len(persistence_dim_0)

In [ ]:
gd.plot_persistence_barcode(persistence_dim_0[-20:])


In [ ]:
plot_intervals_esc(persistence_dim_0[-20:])

In [ ]:
# Crear el complejo simplicial a partir de la nube de puntos
#rips_complex = gd.RipsComplex(points=point_cloud, max_edge_length=1.0)
rips_complex = gd.RipsComplex(distance_matrix=matrix_distance,max_edge_length=promedio)

# Crear el complejo simplicial filtrado
simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

# Calcular la homología persistente
diag = simplex_tree.persistence()

# Mostrar los diagramas de persistencia
gd.plot_persistence_diagram(diag)

In [ ]:
diag

In [ ]:
import gudhi
from scipy.spatial.distance import cdist
points = [[1, 1], [7, 0], [4, 6], [9, 6], [0, 14], [2, 19], [9, 17]]
distance_matrix = cdist(points, points)
cplx = gudhi.SimplexTree.create_from_array(distance_matrix, max_filtration=12.0)
cplx.expansion(2)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

df2=df.iloc[:,1:4]

# Crear una figura 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Agregar los puntos al gráfico
ax.scatter(df['x[cm]'], df['y[cm]'], df['z[cm]'])

# Etiquetas de los ejes
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Título del gráfico
ax.set_title('Nube de Puntos en 3D')

# Mostrar el gráfico
plt.show()


In [ ]:
df_filtered = df[df['rho[cm-3]'] > 1000]
df_filtered

In [ ]:
df_filtered2 = df[df['h[cm]'] > 10000]
df_filtered2

In [ ]:
import plotly.express as px

# Asumiendo que 'df' es tu DataFrame y ya contiene las columnas 'x[cm]', 'y[cm]', 'z[cm]', y 'rho[cm-3]'

fig = px.scatter_3d(df, x='x[cm]', y='y[cm]', z='z[cm]', color='rho[cm-3]',
                    color_continuous_scale=px.colors.sequential.Viridis)

# Actualizar los títulos de los ejes
fig.update_layout(scene=dict(
                    xaxis_title='X[cm]',
                    yaxis_title='Y[cm]',
                    zaxis_title='Z[cm]'),
                  title='Nube de Puntos en 3D con Colores por Densidad')

# Mostrar el gráfico
fig.show()
